# ASSIGNMENT

# Service Enumeration:

In [1]:
%env TARGET=10.11.1.17

env: TARGET=10.11.1.17


# SMB:

In [2]:
!nmap -v -sV --open -Pn  -p 137,139,445 --script=smb-ls $TARGET

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.
Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-27 15:15 PDT
NSE: Loaded 46 scripts for scanning.
NSE: Script Pre-scanning.
Initiating NSE at 15:15
Completed NSE at 15:15, 0.00s elapsed
Initiating NSE at 15:15
Completed NSE at 15:15, 0.00s elapsed
Initiating Parallel DNS resolution of 1 host. at 15:15
Completed Parallel DNS resolution of 1 host. at 15:15, 0.01s elapsed
Initiating Connect Scan at 15:15
Scanning 10.11.1.17 [3 ports]
Discovered open port 445/tcp on 10.11.1.17
Discovered open port 139/tcp on 10.11.1.17
Completed Connect Scan at 15:15, 2.06s elapsed (3 total ports)
Initiating Service scan at 15:15
Scanning 2 services on 10.11.1.17
Completed Service scan at 15:15, 11.22s elapsed (2 services on 1 host)
NSE: Script scanning 10.11.1.17.
Initiating NSE at 15:15
Completed NSE at 15:15, 0.01s elapsed
Initiating NSE at 15:15
Completed NSE at 15:15, 0.00s elapsed
Nmap scan report for 1

In [3]:
!nmap -v -sV --open -Pn  -p 137,139,445 --script=smb-os-discovery,smb-enum-shares,smb-security-mode $TARGET --script-args=smbbasic=1,smbsign=ignore | tee smb_results.txt

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.
Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-27 15:15 PDT
NSE: Loaded 48 scripts for scanning.
NSE: Script Pre-scanning.
Initiating NSE at 15:15
Completed NSE at 15:15, 0.00s elapsed
Initiating NSE at 15:15
Completed NSE at 15:15, 0.00s elapsed
Initiating Parallel DNS resolution of 1 host. at 15:15
Completed Parallel DNS resolution of 1 host. at 15:15, 0.01s elapsed
Initiating Connect Scan at 15:15
Scanning 10.11.1.17 [3 ports]
Discovered open port 445/tcp on 10.11.1.17
Discovered open port 139/tcp on 10.11.1.17
Completed Connect Scan at 15:15, 0.06s elapsed (3 total ports)
Initiating Service scan at 15:15
Scanning 2 services on 10.11.1.17
Completed Service scan at 15:15, 11.21s elapsed (2 services on 1 host)
NSE: Script scanning 10.11.1.17.
Initiating NSE at 15:15
Completed NSE at 15:15, 6.97s elapsed
Initiating NSE at 15:15
Completed NSE at 15:15, 0.00s elapsed
Nmap scan report for 1

In [4]:
import os

try:
    result = !cat smb_results.txt | grep account_used

    anon_user = result[0].split(':')[-1].strip()
except:
    anon_user = "guest"

if anon_user == '<blank>':
    anon_user = "guest"
anon_user

'guest'

In [5]:
!~/hax/enum4linux.pl -a -u {anon_user} {os.environ['TARGET']}

Starting enum4linux v0.9.1 ( http://labs.portcullis.co.uk/application/enum4linux/ ) on Wed Oct 27 15:15:58 2021

 =========================================( Target Information )=========================================

Target ........... 10.11.1.17
RID Range ........ 500-550,1000-1050
Username ......... 'guest'
Password ......... ''
Known Usernames .. administrator, guest, krbtgt, domain admins, root, bin, none


 =============================( Enumerating Workgroup/Domain on 10.11.1.17 )=============================


[+] Got domain/workgroup name: WORKGROUP


 =================================( Nbtstat Information for 10.11.1.17 )=================================

Looking up status of 10.11.1.17
	PBX             <00> -         B <ACTIVE>  Workstation Service
	PBX             <03> -         B <ACTIVE>  Messenger Service
	PBX             <20> -         B <ACTIVE>  File Server Service
	WORKGROUP       <00> - <GROUP> B <ACTIVE>  Domain/Workgroup Name
	WORKGROUP       <1e> - <GROUP> B <A

In [6]:
!~/hax/enum4linux.pl $TARGET

Starting enum4linux v0.9.1 ( http://labs.portcullis.co.uk/application/enum4linux/ ) on Wed Oct 27 15:16:07 2021

 =========================================( Target Information )=========================================

Target ........... 10.11.1.17
RID Range ........ 500-550,1000-1050
Username ......... ''
Password ......... ''
Known Usernames .. administrator, guest, krbtgt, domain admins, root, bin, none


 =============================( Enumerating Workgroup/Domain on 10.11.1.17 )=============================


[+] Got domain/workgroup name: WORKGROUP


 =================================( Nbtstat Information for 10.11.1.17 )=================================

Looking up status of 10.11.1.17
	PBX             <00> -         B <ACTIVE>  Workstation Service
	PBX             <03> -         B <ACTIVE>  Messenger Service
	PBX             <20> -         B <ACTIVE>  File Server Service
	WORKGROUP       <00> - <GROUP> B <ACTIVE>  Domain/Workgroup Name
	WORKGROUP       <1e> - <GROUP> B <ACTIVE

In [7]:
!~/hax/enum4linux.pl -u administrator $TARGET 

Starting enum4linux v0.9.1 ( http://labs.portcullis.co.uk/application/enum4linux/ ) on Wed Oct 27 15:23:44 2021

 =========================================( Target Information )=========================================

Target ........... 10.11.1.17
RID Range ........ 500-550,1000-1050
Username ......... 'administrator'
Password ......... ''
Known Usernames .. administrator, guest, krbtgt, domain admins, root, bin, none


 =============================( Enumerating Workgroup/Domain on 10.11.1.17 )=============================


[+] Got domain/workgroup name: WORKGROUP


 ====================================( Session Check on 10.11.1.17 )====================================


[+] Server 10.11.1.17 allows sessions using username 'administrator', password ''


 =================================( Getting domain SID for 10.11.1.17 )=================================

Bad SMB2 signature for message
[0000] 00 00 00 00 00 00 00 00   00 00 00 00 00 00 00 00   ........ ........
[0000] 0B 72 F3 EF

In [8]:
!~/hax/enum4linux.pl -u guest $TARGET 

Starting enum4linux v0.9.1 ( http://labs.portcullis.co.uk/application/enum4linux/ ) on Wed Oct 27 15:23:45 2021

 =========================================( Target Information )=========================================

Target ........... 10.11.1.17
RID Range ........ 500-550,1000-1050
Username ......... 'guest'
Password ......... ''
Known Usernames .. administrator, guest, krbtgt, domain admins, root, bin, none


 =============================( Enumerating Workgroup/Domain on 10.11.1.17 )=============================


[+] Got domain/workgroup name: WORKGROUP


 ====================================( Session Check on 10.11.1.17 )====================================


[+] Server 10.11.1.17 allows sessions using username 'guest', password ''


 =================================( Getting domain SID for 10.11.1.17 )=================================

Bad SMB2 signature for message
[0000] 00 00 00 00 00 00 00 00   00 00 00 00 00 00 00 00   ........ ........
[0000] 73 2D 73 F8 B6 00 F7 C7   4

In [9]:
!~/hax/enum4linux.pl -u john $TARGET 

Starting enum4linux v0.9.1 ( http://labs.portcullis.co.uk/application/enum4linux/ ) on Wed Oct 27 15:23:47 2021

 =========================================( Target Information )=========================================

Target ........... 10.11.1.17
RID Range ........ 500-550,1000-1050
Username ......... 'john'
Password ......... ''
Known Usernames .. administrator, guest, krbtgt, domain admins, root, bin, none


 =============================( Enumerating Workgroup/Domain on 10.11.1.17 )=============================


[+] Got domain/workgroup name: WORKGROUP


 ====================================( Session Check on 10.11.1.17 )====================================


[+] Server 10.11.1.17 allows sessions using username 'john', password ''


 =================================( Getting domain SID for 10.11.1.17 )=================================

Bad SMB2 signature for message
[0000] 00 00 00 00 00 00 00 00   00 00 00 00 00 00 00 00   ........ ........
[0000] 36 C8 5C A8 CB 5E 94 E8   F4 

In [ ]:
!smbclient -L $TARGET -U guest

Enter WORKGROUP\guest's password: 

In [ ]:
%env SHARE=Users

In [ ]:
!echo Run this:
!echo cd \'$(pwd)\'
!echo smbclient //$TARGET/$SHARE -U guest

In [ ]:
from web_footprinting_utils import show_term

show_term()

#### Use ls, get, cd to pull down any needed files in SMB...

In [ ]:
!ls

In [ ]:
!cat log.txt

# NFS Anonymous Mounting:

In [ ]:
from getpass import getpass
import os

%env RDIR = /var

In [ ]:
# !mkdir -p my_mount
# !echo { getpass() } | sudo -S sudo mount {os.environ['TARGET']}:{os.environ['RDIR']} my_mount
# !ls -la my_mount

# SQL Map:

In [ ]:
!sqlmap -r bugle_user.txt --dbms=mariadb --dump --batch

In [ ]:
!sqlmap -r bugle_admin.txt --dbms=mariadb --dump --batch

In [ ]:
!sqlmap -r bugle_admin.txt --dbms=mariadb --dump --batch --level=5

# SQL Login:

In [ ]:
mysql -h 127.0.0.1 -u techblog -p'z8n#DZf@Sa#X!4@tqG'

# FTP:

ftp 10.10.28.39

anonymous

no password

#### remember to enable binary mode to download binaries:

binary